In [21]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.llms.openai import OpenAI
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [23]:
class RAG_agent:
    def __init__(self, index_path):
        storage_context = StorageContext.from_defaults(persist_dir=index_path)
        index = load_index_from_storage(storage_context)
        llm = OpenAI(
            model="gpt-4o",
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0.2,
            num_outputs=1000,
        )
        self.query_engine = index.as_query_engine(response_mode="refine", similarity_top_k=5, verbose = True, llm = llm)

    def query(self, query):
        return self.query_engine.query(query)


In [24]:
class Query_agent:
    # This agent takes user prompt and refine to a query for specific RAG agent
    def __init__(self, description, history=[]):
        self.client = OpenAI() # TODO openai.OpenAI()
        self.messages = history + [
            {"role": "system", "content": description},
            {"role": "user", "content": ""}
        ]

    def create_query(self, prompt):
        self.messages[1]["content"] = prompt
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=self.messages,
            max_tokens=1000,
            n=1,
            top_p=0.95,
            frequency_penalty=0.2,
            temperature=0.7
        )
        return response.choices[0].message.content
        

In [25]:
class Response_agent:
    # This agent takes the history and create the final response to user
    def __init__(self, description, history):
        self.client = OpenAI() # TODO openai.OpenAI()
        self.messages = history
        self.description = [
            {"role": "system", "content": description},
        ]

    def respond(self):
        print(self.messages)
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=self.messages,
            max_tokens=1000,
            n=1,
            top_p=0.95,
            frequency_penalty=0.2,
            temperature=0.7
        )
        return response.choices[0].message.content

In [26]:
strategy_query_description = """You are a Strategy Query Refinement Agent.
Your primary function is to interpret the user's prompts related to 
gameplay strategies, character builds, or weapon recommendations in Elden Ring. 
You focus on understanding the user's goals, preferences, and challenges. You 
then refine and formulate an effective query tailored to retrieve relevant 
information from the "YouTuber Strategy and Weapon Recommendations" vector store. 
By crafting precise queries, you enhance the RAG process, ensuring that the most 
pertinent strategic insights are fetched on behalf of the user."""

In [27]:
gameinfo_query_description = """You are an In-Game Data Query Refinement Agent.
Specializing in in-game data retrieval, you analyze the user's requests for 
specific information about items, such as locations, weights, prices, or stats 
within Elden Ring. Your role is to comprehend the exact details the user is 
seeking and refine the query accordingly. You construct an optimized search query 
to access the required information from the "In-Game Data" vector store sourced 
from the Elden Ring wiki. By focusing on precise query formulation, you facilitate 
the RAG process to deliver accurate in-game data on behalf of the user."""

In [28]:
final_response_description = """You are an intelligent response agent tasked with 
generating clear and helpful answers to the user's initial prompts by effectively 
utilizing the query results obtained from the RAG process. With access to the 
complete message history—including the user's original question and the information 
retrieved from both the "YouTuber Strategy and Weapon Recommendations" and "In-Game 
Data" vector stores—you analyze the relevance and accuracy of the retrieved 
information. Your goal is to construct informed responses that integrate pertinent 
data to provide comprehensive and useful answers. If the query results are 
irrelevant or insufficient, you recognize this and avoid confusing the user, opting 
instead to inform them that specific information is not available or to ask 
clarifying questions. By intelligently handling both relevant and irrelevant query 
results, you ensure the user receives meaningful assistance without unnecessary 
confusion, enhancing their experience with the Elden Ring game guide system."""

In [29]:
class EldenGuideSystem:
    def __init__(self, strategy_agent, gameinfo_agent):
        self.messages = []
        self.strategy_agent = strategy_agent
        self.gameinfo_agent = gameinfo_agent
        self.strategy_query_agent = Query_agent(strategy_query_description)
        self.gameinfo_query_agent = Query_agent(gameinfo_query_description)
        self.final_response_agent = Response_agent(final_response_description, self.messages)

    def push_assistant_message(self, message):
        self.messages.append({
            "role": "assistant",
            "content": message
        })

    def run(self, prompt):
        strategy_query = self.strategy_query_agent.create_query(prompt)
        strategy_response = self.strategy_agent.query(strategy_query)
        self.push_assistant_message(strategy_response)
        gameinfo_query = self.gameinfo_query_agent.create_query(prompt)
        gameinfo_response = self.gameinfo_agent.query(gameinfo_query)
        self.push_assistant_message(gameinfo_response)
        print(self.final_response_agent.messages)
        return "test"
        # final_response = self.final_response_agent.respond()
        # return final_response


In [ ]:
# index paths
strategy_path = "index/strategy"
gameinfo_path = "index/game"
altogether_path = "index/altogether"

In [30]:
# read prompt from csv
import pandas as pd
df = pd.read_csv("prompt.csv")
# get first row
prompt = df.iloc[0, 0]
print(prompt)


What is the best type of upgrade for Nagakiba to cause the most damage?


In [31]:
strategy_agent = RAG_agent(strategy_path)
gameinfo_agent = RAG_agent(gameinfo_path)

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Users/admin/Desktop/1786/EldenBossBuster/../index/strategy/docstore.json'

In [ ]:

system = EldenGuideSystem(strategy_agent, gameinfo_agent)
response = system.run(prompt)
print(response)